# Modelado semántico
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

El modelado léxico tiene importantes usos en la recuperación de información; los sistemas pueden ser altamente escalables si hay una implementación adecuada en índice invertido. Sin embargo, sufren de diversos problemas debido a su naturaleza:

- Las consultas deben ser expresadas con los mismos términos que los documentos.
- Los términos con múltiples significados pueden introducir falsos positivos en los resultados.
- En general se tienen problemas con la sinónimia, la homonimia, y la hiponimia, así como las variaciones léxicas resultantes de errores ortográficos, ya sean intencionados o involuntarios.

Aunque existen soluciones dos grandes vertientes de modelado semántico:
- Modelado del vocabulario: Intententa asignar cada palabra del vocabulario a un _concepto_ o significado _semántico_, que esta representado por un vector de alta dimensión (embedding).
- Modelado del lenguaje: Intenta modelar el lenguaje, normalmente aprendiendo que palabras pueden seguir a un contexto dado. La diferencia principal al modelado de vocabulario es que cuando se modela el lenguaje se tiene un modelo sensible al contexto, y por lo tanto, para una misma palabra, el contexto completo derminará su vector _embedding_. 

Aunque existen diversos intentos de modelado semántico en la literatura clásica, hasta tiempos recientes es cuando ha tomado gran interés debido a los impresionantes avances en redes neuronales profundas que han permitido aprender estos modelos de enormes colecciones, capturando una semántica profunda. Los grandes modelos de lenguaje han abierto nuevas posibilidades sobre las aplicaciones posibles en recuperación de información.

## Sobre las representaciones de documentos
Los modelos semánticos de palabras, anteriormente discutidos, son insuficientes para la representación de documentos; para conseguir representaciones de documentos hay diferentes estrategias. Por ejemplo, es posible generar combinaciones lineales de los embeddings individuales para generar vector embedding de documento. Algunos modelos basados en redes neuronales, generan embeddings de documento por diseño.


## Organización del resto de la unidad
Esta únidad esta dividida en dos notebooks. El primero (este documento) está dedicado al modelado semántico de vocabularios, mientras que el segundo se enfoca a modelos de lenguaje que trabajen sobre documentos. 

# Modelado semántico de vocabularios
El modelado basado en vocabularios o léxico tiene una gran cantidad de aplicaciones, las cuales son suelen explotar la gran cantidad de grados de libertad en el preprocesamiento, toquenizado y pesado, para ajustar a las tareas deseadas. Sin embargo, dicho modelado tiene problemas basados en la semántica:

- si las consultas tienen palabras no conocidas (fuera de vocabulario), no es posible ligar a palabras similares por semántica 
- las palabras similares en el vocabulario pero no iguales, no puede relacionarse para reducir un vocabulario, o para cualquier otro efecto
- las palabras que se escriben igual pero que tienen significado idéntico serán puestas en la misma bolsa

De manera más detallada, se tendrán problemas al manejar sinónimos, hipónimos, antónimos, etc. Adicionalmente, los errores serán tratados como palabras diferentes y no ajustados al mejor calce conocido (a menos que exista un corrector ortográfico entre el preprocesamiento).

Estos problemas se buscan reducir con representaciones semánticas de los vocabularios y de los textos, esto sería equivalente. El intento sería buscar por conceptos más que por palabras que lo describan.
En este punto aparecen los modelos semánticos, los cuales suelen ser representaciones vectoriales densas _embeddings_, de alta dimensión pero mucho menor que las representaciones léxicas (i.e., pasarían a unos pocos cientos en lugar de varias decenas de miles o cientos de miles de coordinadas).

# Word embeddings
Las representaciones de vectores densos de las palabras se suelen construir utilizando la [_hipótesis distribucional_](https://aclweb.org/aclwiki/Distributional_Hypothesis), que se puede resumir que las palabras con contextos similares (palabras que la rodean) tienen un significado similar. Entonces, para obtener la semántica, es necesario comparar contextos; lo anterior se puede llevar a cabo de diferentes maneras.

- Latent semantic analysis (LSA): La matriz $W_{m,n}^*$  con pesos basados en frecuencias es usada para reducir el numero de filas (palabras) mediante _Singular Value Decomposition_ (SVD). La similitud adecuada entre vectores en el espacio de dimensión reducida es el coseno. Ver [@Dumais2004] para más información.
- Latent semantic indexing (LSI): Similar a LSA pero usando esquemas de pesado diferentes, adicionalmente se usa Trucated SVD (TSVD) en lugar de SVD ya que puede seleccionar componentes más importantes. Para más detalles ver [@Hofman1999].
- Word2Vec. Se construye una red neuronal con una matriz de pesos del tamaño del vocabulario por una dimensión seleccionada, a partir de los contextos. Revisa los textos de entrenamiento mediante una ventana móvil. De manera especial, se tiene que el centro de la ventana es la palabra objetivo y el contexto esta a su alrededor. Se intenta predecir mediante el contexto la palabra central, y se ajusta mediante propagación hacia atrás. La función de perdida suele ser [softmax](https://en.wikipedia.org/wiki/Softmax_function) o hierachical softmax. Los embeddings no son la capa de salida sino una capa atrás. Para más detalles ver [@MCCD2013].
- Glove. Se utiliza una matriz de coocurrencia de términos dentro de una ventana deslizante que analiza una colección de textos. La matriz es altamente dispersa y de alta dimensión. Plantea un problema de optimización lineal para capturar la información más útil y bajar su dimensión. Los detalles se pueden consultar en [@PSM2014]. 
- FastText. Muy similar a Word2Vec pero añade manejo de palabras fuera de diccionario mediante subwords (similar a q-gramas de carácteres por cada palabra). Ver [@BGJM2017] para mayores detalles.

## Nota
Este notebook esta basado en <https://github.com/sadit/SimilaritySearchDemos/blob/main/Glove/Glove.ipynb>. En el notebook original se muestran visualizaciones basadas en todos los vecinos cercanos (se verán más adelante en el curso).

# Ejemplo

El resto del documento muestra como usar word embeddings Glove para precalculados, para resolver tareas de vecinos cercanos y resolución de analogías.

In [2]:
using Pkg
Pkg.activate(".")
using SimilaritySearch, Plots, LinearAlgebra, Embeddings, HypertextLiteral

  Activating project at `~/Cursos/IR-2024/Unidades`


In [3]:
function load_dataset()
    emb = load_embeddings(GloVe, 2)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end

    db = MatrixDatabase(emb.embeddings)
    db, emb.vocab
end

function create_index()
    db, vocab = load_dataset()
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db)
    index!(index)
    optimize_index!(index, MinRecall(0.9))
    index, vocab
end


create_index (generic function with 1 method)

### El índice métrico se crea, además se toma el vocabulario

In [4]:
@time index, vocab = create_index()
vocab2id = Dict(w => i for (i, w) in enumerate(vocab));

append_items! sp=3084, ep=3340, n=3340, BeamSearch
  bsize: Int32 5
  Δ: Float32 1.0640187f0
  maxvisits: Int64 520
, 2024-01-10T13:21:36
append_items! sp=21331, ep=21587, n=21587, BeamSearch
  bsize: Int32 10
  Δ: Float32 1.073014f0
  maxvisits: Int64 860
, 2024-01-10T13:21:38.296
append_items! sp=74530, ep=74786, n=74786, BeamSearch
  bsize: Int32 15
  Δ: Float32 1.1f0
  maxvisits: Int64 1186
, 2024-01-10T13:21:43.982
append_items! sp=88408, ep=88664, n=88664, BeamSearch
  bsize: Int32 52
  Δ: Float32 1.25939f0
  maxvisits: Int64 4692
, 2024-01-10T13:21:45.615
append_items! sp=94833, ep=95089, n=95089, BeamSearch
  bsize: Int32 52
  Δ: Float32 1.25939f0
  maxvisits: Int64 4692
, 2024-01-10T13:21:46.421
append_items! sp=118734, ep=118990, n=118990, BeamSearch
  bsize: Int32 52
  Δ: Float32 1.25939f0
  maxvisits: Int64 4692
, 2024-01-10T13:21:49.611
append_items! sp=181699, ep=181955, n=181955, BeamSearch
  bsize: Int32 38
  Δ: Float32 1.1235001f0
  maxvisits: Int64 3310
, 2024-01-10T1

 58.894566 seconds (8.52 M allocations: 3.536 GiB, 1.38% gc time, 4.80% compilation time: 1% of which was recompilation)


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

In [5]:
let k = 32
    @time iI, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    @time gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    macrorecall(gI, iI)
end

  7.832779 seconds (1.52 M allocations: 178.705 MiB, 3.69% gc time, 10.29% compilation time)
271.558314 seconds (323.74 k allocations: 117.807 MiB, 0.12% compilation time)


0.9079178125

### Búsqueda y presentación de los resultados

In [9]:
function search_and_display(index, vocab, q, res, k, qword)
    res = reuse!(res, k)
    @time search(index, q, res)

    L = []
    for (j, p) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$(vocab[p.id])</td><td>$(p.id)</td><td>$(round(p.weight, digits=3))</td></tr>")
    end

    display(@htl """<h2>resultados for "$qword"</h2>
    <table>
    <th>  <td>word</td> <td>id</td> <td>dist</td> </th>
        $L
    </table>
    """)
end

function search_and_display(index, vocab, qid::Integer, res, k=maxlength(res))
    search_and_display(index, vocab, index[qid], res, k, vocab[qid])
end

display(@htl "<h1>Ejemplos de búsqueda (aleatorios)</h1>")
res = KnnResult(7)
for i in 1:3
    for qid in rand(1:length(vocab))
        search_and_display(index, vocab, qid, res)
    end
end

<h1>Ejemplos de búsqueda (aleatorios)</h1>

  0.000597 seconds (3 allocations: 128 bytes)


1,trackman,358840,0.0
2,k-factor,354521,0.273
3,calyptra,342308,0.284
4,triquetra,360038,0.296
5,two-layer,367979,0.302
6,soteria,334357,0.306
7,wubi,329119,0.307


  0.000276 seconds (3 allocations: 128 bytes)


1,10-step,331786,0.0
2,counter-proposal,374976,0.268
3,u.s.-orchestrated,345590,0.271
4,u.s.-inspired,231938,0.271
5,photo-shoot,336587,0.275
6,lightscribe,379165,0.285
7,governmentwide,261898,0.29


  0.000586 seconds (3 allocations: 128 bytes)


1,phillinganes,387519,0.0
2,proops,238001,0.257
3,luzinski,237569,0.276
4,kurstin,172215,0.279
5,leisz,283755,0.28
6,keagle,292174,0.289
7,cadaret,275195,0.316


In [10]:
search_and_display(index, vocab, vocab2id["glove"], res, 15)

  0.000193 seconds (3 allocations: 128 bytes)


1,glove,10923,0.0
2,ball,1084,0.369
3,gloves,10808,0.369
4,plate,4365,0.379
5,pads,15978,0.418
6,infield,16731,0.435
7,bounced,9154,0.435
8,hat,5626,0.438
9,bat,4926,0.439
10,fielder,10241,0.447
11,pitch,3099,0.45


# Analogias

In [11]:
function analogy(a, b, c, k)
	v = index[vocab2id[a]] - index[vocab2id[b]] + index[vocab2id[c]]
	normalize!(v)
	search_and_display(index, vocab, v, res, k, "<$a> - <$b> + <$c>")
end

analogy("father", "man", "woman", 5)
analogy("fireman", "man", "woman", 5)
analogy("mississippi", "usa", "france", 5)

  0.000061 seconds (3 allocations: 128 bytes)


1,mother,809,0.098
2,daughter,1132,0.132
3,wife,703,0.147
4,father,630,0.148
5,husband,1328,0.172


  0.000096 seconds (3 allocations: 128 bytes)


1,fireman,27345,0.156
2,firefighter,15812,0.303
3,paramedic,33841,0.393
4,rescuer,44915,0.439
5,passerby,53776,0.459


  0.000116 seconds (3 allocations: 128 bytes)


1,rhine,13957,0.488
2,coast,955,0.506
3,brittany,15877,0.506
4,southern,483,0.506
5,northern,530,0.513


# Actividades 
- Como se menciono, el uso de redes neuronales se vuelve muy popular para aprender la semántica desde el texto. Comente las razones justificando sus comentarios en términos de aprendizaje y requerimientos computacionales.
- Discuta que tipo de métrica se usa y porque.
- ¿Cómo se podrían usar los word-embeddings para búsqueda de documentos completos?
- Como ajustaría la métrica.
- Si usa Julia, revisé el paquete `Embeddings.jl`, `Word2Vec.jl`.
- Si usa Python, revisé los paquetes `fastText` y `word2vec`.
- Glove <https://nlp.stanford.edu/projects/glove/>
- Reproduzca el ejercicio de este notebook (no reproduzca la búsqueda exhaustiva, ya que le tomará muchísimo tiempo), use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Anote sus reflexiones.

# Bibliografía
- [Dumais2004] Dumais, S. T. (2004). Latent semantic analysis. Annu. Rev. Inf. Sci. Technol., 38(1), 188-230.
- [Hofmann1999] Hofmann, T. (1999, August). Probabilistic latent semantic indexing. In Proceedings of the 22nd annual international ACM SIGIR conference on Research and development in information retrieval (pp. 50-57).
- [MCCD2013]: Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781.
- [PSM2014] Pennington, J., Socher, R., & Manning, C. D. (2014, October). Glove: Global vectors for word representation. In Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP) (pp. 1532-1543).
- [BGJM2017]: Bojanowski, P., Grave, E., Joulin, A., & Mikolov, T. (2017). Enriching word vectors with subword information. Transactions of the association for computational linguistics, 5, 135-146.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.